In [1]:
import os
from openai import OpenAI
from pydantic import BaseModel
import json

In [2]:
client = OpenAI(
    api_key = os.getenv('OPEN_AI_API')
)

In [3]:
#read jsonl into array
attractions = []

# Open the file and read it line by line
with open('Datasets/Attractions_pruned.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append to the list
        json_obj = json.loads(line.strip())  # strip() removes any leading/trailing whitespace
        attractions.append(json_obj)


#read jsonl into array
restaurants = []

# Open the file and read it line by line
with open('Datasets/Restaurants_pruned.jsonl', 'r') as file:
    for line in file:
        # Parse each line as a JSON object and append to the list
        json_obj = json.loads(line.strip())  # strip() removes any leading/trailing whitespace
        restaurants.append(json_obj)


In [4]:
class OpenCat_attractions(BaseModel):
    attraction_type: str
    perks: str

In [5]:
class OpenCat_restaurants(BaseModel):
    restaurant_type_1: str
    restaurant_type_2: str
    cuisine_1: str
    cuisine_2: str
    perks: str

In [6]:
def llm(system_prompt,user_prompt,format):    
    
    #generation into json format
    chat_completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system","content": system_prompt,},
            {"role": "user","content": user_prompt,}
        ],
        response_format=format
    )

    return json.loads(chat_completion.choices[0].message.parsed.json())

In [8]:
with open('Prompts/Open Categories Prompts/Attraction_open_cat.txt', 'r') as file:
    system_prompt = file.read()

In [9]:
results = []
for i in range(len(attractions)):
    business_id = attractions[i]['business_id']
    
    user_prompt = str(attractions[i])

    result = llm(system_prompt,user_prompt,OpenCat_attractions)
    result['business_id'] = business_id
    results.append(result)

In [11]:
#write into jsonl file
with open('Datasets/Attractions_open_cat.jsonl', 'w') as json_file:
    for item in results:
        json.dump(item, json_file)
        json_file.write('\n') 

In [48]:
with open('../Prompts/Open Categories Prompts/Restaurants_open_cat.txt', 'r') as file:
    system_prompt = file.read()

In [49]:
results = []
for i in range(len(restaurants)):
    business_id = restaurants[i]['business_id']
    
    user_prompt = str(restaurants[i])

    result = llm(system_prompt,user_prompt,OpenCat_restaurants)
    result['business_id'] = business_id
    results.append(result)

In [50]:
#write into jsonl file
with open('../Datasets/Restaurants_open_cat.jsonl', 'w') as json_file:
    for item in results:
        json.dump(item, json_file)
        json_file.write('\n') 